In [1]:
import pandas as pd
import numpy as np
from scipy import stats

from sklearn.model_selection import train_test_split , cross_val_score
from sklearn.metrics import r2_score , mean_squared_error
from sklearn.metrics import accuracy_score ,mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

# Load the data

### Testing on 10000 data is time Consuming so better to test on 1000 data set and if prediction is best then change data size

In [2]:
x = pd.read_csv("../tsrl10000.csv",nrows = 10000)
x = x.iloc[:,1:]
y = pd.read_csv("../srlbininfo_10000",sep ="\t",header =None,nrows = 10000)
y = pd.DataFrame(y.iloc[:,1])
y.columns = ["RL"]

df = pd.concat([y,x],axis =1)

y = pd.DataFrame(df.loc[:,"RL"])
x = df.iloc[:,1:]  #    

#Caution X Column up to 8000 above that outlier and y.GC Less than 

In [34]:
y.head()

,RL
0,19.0
1,18.0
2,16.0
3,15.0
4,18.0


## Always Check the data shape , df , x ,y

In [4]:
df.shape,x.shape , y.shape

((10000, 3881), (10000, 3880), (10000, 1))

In [5]:
display(x.head(2))
display(y.head(2))

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V3871,V3872,V3873,V3874,V3875,V3876,V3877,V3878,V3879,V3880
0,1.179333,1.334434,1.403288,1.541632,1.325201,1.101396,1.096208,1.169582,1.201159,1.145272,...,0,0,0,0,0,0,0,0,0,0
1,1.148473,1.422455,1.373932,1.515307,1.337463,1.077379,1.061001,1.157020,1.258965,1.140321,...,0,0,0,0,0,0,0,0,0,0


,RL
0,19.0
1,18.0


In [6]:
x_train= None;x_test = None;y_train = None;y_test = None
### 5 Regression 
clf_ridge = None ; y_pred_ridge = None;
clf_lasso = None ; y_pred_lasso = None
regr_elasticNet= None ; y_pred_elasticNet = None
regr_enetcv = None ;y_pred_elasticnetcv = None 
clf_bay = None ; y_pred_bay = None ;

In [7]:
def train_test(size = 0.20 , random_state=50,shuffle = True):
    "size = train size and random state is how the data are mixed and shuffle in to do mixing or not  "
    from sklearn.model_selection import train_test_split
    global x_train,x_test,y_train, y_test
    x_train= None; x_test = None; y_train = None; y_test = None
    x_train,x_test,y_train, y_test = train_test_split(x,y,test_size= size,random_state = random_state , shuffle = shuffle)
    
    #return x_train,x_test,y_train, y_test 

In [8]:
#train_test() # Calling The Function with default Parameter
#print(y_train.shape)
#display(y_train.head(2))
#print(x_train.shape)
#display(x_train.head())

##  Create a Function for Lasso

In [9]:
#Create a Lasso Model to automatically apply in the loop

def lasso( alpha=0.5, fit_intercept=True, normalize=False, precompute=False, copy_X=True, 
                      max_iter=1000, tol=0.0001, warm_start=False, positive=False,selection='cyclic'):
    
    "Just Pass Related values only"
    
    from sklearn.linear_model import Lasso
    from sklearn.model_selection import train_test_split
    global x_train,x_test,y_train, y_test , clf_lasso , y_pred_lasso
    
    clf_lasso = Lasso(alpha=alpha, fit_intercept=fit_intercept , normalize=normalize, precompute=precompute, copy_X=copy_X, 
                      max_iter=max_iter, tol=tol, warm_start=warm_start, positive=positive,selection=selection).fit(x_train,y_train)
    
    y_pred_lasso = pd.DataFrame(clf_lasso.predict(x_test))
    name = "Lasso"
    # *****************************************************************
    
#    print(name +" Coefficient ", clf_lasso.coef_) 
#    print(name + " Intercept " , clf_lasso.intercept_,"\n") ##
#    print("****"*5+"Accuracy Test Model Fitting "+"****"*5)
#    print(name + "Score for train data set :",clf_lasso.score(x_train,y_train))
#    print(name + "Score for test data Set",clf_lasso.score(x_test,y_test))
#
#    print(name + "Score for Predictecd data Set",clf_lasso.score(x_test,y_pred_lasso)) ##
#    
#    #print("\n"+"****"*5+"R2 Score"+"****"*5)
#    #print(name + "R2 Score for test is = ", r2_score(y_test, y_pred_lasso)) ##
#    
#    print("\n"+"****"*5+"Mean Absolute Error"+"****"*5)
#    print(name+ "Mean Absolute Error of Test = ", mean_absolute_error(y_test,y_pred_lasso)) ##
#    print(name +"Mean Absolute Error on Train",mean_absolute_error(y_train,pd.DataFrame(clf_lasso.predict(x_train))))
#    
#    print("\n"+"****"*5+" Mean Squared Error"+"****"*5)
#    print(name + "Mean Squared Error of Test = ",mean_squared_error(y_test,y_pred_lasso)) ##
#    print(name+"Mean Squared Error on Train = ",mean_squared_error(y_train,pd.DataFrame(clf_lasso.predict(x_train))))
#    
#    print("\n"+"****"*5+" Co Relation"+"****"*5)
#    print(name + "Correlation Test Data ",stats.pearsonr(y_test["RL"],y_pred_lasso[0])[0])
#    print(name + "Correlation Original Data ",stats.pearsonr(y_train["RL"],pd.DataFrame(clf_lasso.predict(x_train))[0])[0])
    
    return clf_lasso.score(x_test,y_test) ,stats.pearsonr(y_test.RL,pd.DataFrame(clf_lasso.predict(x_test))[0])[0]

## Function For Ridge Regression

In [10]:
#Create a Model to automatically apply in the loop

def ridge( alpha = 0.01,copy_X=True,fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001):
    "Just Pass alpha value only"
    
    from sklearn.linear_model import Ridge
    from sklearn.model_selection import train_test_split
    global x_train,x_test,y_train, y_test , clf_ridge , y_pred_ridge
    
    clf_ridge = Ridge(alpha=alpha,copy_X=copy_X,fit_intercept=fit_intercept, max_iter=max_iter, normalize=normalize,
      random_state=random_state, solver=solver, tol=tol).fit(x_train,y_train)
    
    y_pred_ridge = pd.DataFrame(clf_ridge.predict(x_test))
    name = "Ridge"
    # *****************************************************************
    
   # print(name +" Coefficient ", clf_ridge.coef_) 
#    print(name + " Intercept " , clf_ridge.intercept_,"\n") ###
#    print("****"*5+"Accuracy Test Model Fitting "+"****"*5)
#    print(name + "Score for Train data set :",clf_ridge.score(x_train,y_train))
#    print(name + "Score for Test data Set",clf_ridge.score(x_test,y_test))
    
    #print(name + "Score for Train data set :",clf_ridge.score(x_train,y_train),"Test data Set =",clf_ridge.score(x_test,y_test))
    #Return the coefficient of determination R^2 of the prediction.
    
    

#    print(name + "Score for Predictecd data Set",clf_ridge.score(x_test,y_pred_ridge)) ##
    
    #print("\n"+"****"*5+"R2 Score"+"****"*5)
    #print(name + "R2 Score for test is = ", r2_score(y_test, y_pred_ridge)) ##
    
#    print("\n"+"****"*5+"Mean Absolute Error"+"****"*5)
#    print(name+ "Mean Absolute Error of Test = ", mean_absolute_error(y_test,y_pred_ridge)) ##
#    print(name +"Mean Absolute Error on Train",mean_absolute_error(y_train,pd.DataFrame(clf_ridge.predict(x_train))))
    
#    print("\n"+"****"*5+" Mean Squared Error"+"****"*5)
#    print(name + "Mean Squared Error of Test = ",mean_squared_error(y_test,y_pred_ridge)) ##
#    print(name+"Mean Squared Error on Train = ",mean_squared_error(y_train,pd.DataFrame(clf_ridge.predict(x_train))))
    
#    print("\n"+"****"*5+" Co Relation"+"****"*5)
#    print(name + "Correlation Test Data ",stats.pearsonr(y_test["RL"],y_pred_ridge[0])[0])
#    print(name + "Correlation Original Data ",stats.pearsonr(y_train["RL"],pd.DataFrame(clf_ridge.predict(x_train))[0])[0])
    
    #print("Correlation = ",stats.pearsonr(y_train["GC"],pd.DataFrame(clf_ridge.predict(x_train))[0])[0])
    
    return  clf_ridge.score(x_test,y_test) ,stats.pearsonr(y_test.RL,pd.DataFrame(clf_ridge.predict(x_test))[0])[0] # clf_ridge ,

## Create A Function for ElasticNet

In [11]:
def enet(alpha=1.0, l1_ratio=0.5, fit_intercept=True, normalize=False,precompute=False, max_iter=1000,
         copy_X=True, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='cyclic'):
    
    from sklearn.linear_model import ElasticNet
    global x_train,x_test,y_train, y_test , regr_elasticNet, y_pred_elasticNet

    name = "Elastic net--> "

    regr_elasticNet = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, fit_intercept=fit_intercept, normalize=normalize,
                                 precompute=precompute, max_iter=max_iter,copy_X=copy_X, tol=tol, warm_start=warm_start, 
                                 positive=positive, random_state=random_state, selection=selection)
    
    regr_elasticNet.fit(x_train , y_train)

    y_pred_elasticNet =pd.DataFrame(regr_elasticNet.predict(x_test))
    
#    print(name +" Coefficient ", regr_elasticNet.coef_) 
#    print(name + " Intercept " , regr_elasticNet.intercept_,"\n") ##
#    print("****"*5+"Accuracy Test Model Fitting "+"****"*5)
#    print(name + "Score for train data set :",regr_elasticNet.score(x_train,y_train))
#    #Return the coefficient of determination R^2 of the prediction.
#    print(name + "Score for test data Set",regr_elasticNet.score(x_test,y_test))
#    print(name + "Score for Predictecd data Set",regr_elasticNet.score(x_test,y_pred_elasticNet)) ##
#    #print("\n"+"****"*5+"R2 Score"+"****"*5)
#    #print(name + "R2 Score for test is = ", r2_score(y_test,y_pred_sgd)) ##
#    print("\n"+"****"*5+"Mean Absolute Error"+"****"*5)
#    print(name+ "Mean Absolute Error of Test = ", mean_absolute_error(y_test,y_pred_elasticNet)) ##
#    print(name +"Mean Absolute Error on Train",mean_absolute_error(y_train,pd.DataFrame(regr_elasticNet.predict(x_train))))
#    print("\n"+"****"*5+" Mean Squared Error"+"****"*5)
#    print(name + "Mean Squared Error of Test = ",mean_squared_error(y_test,y_pred_elasticNet)) ##
#    print(name+"Mean Squared Error on Train = ",mean_squared_error(y_train,pd.DataFrame(regr_elasticNet.predict(x_train))))
#    print("\n"+"****"*5+" Co Relation"+"****"*5)
#    print(name + "Correlation Test Data ",stats.pearsonr(y_test["RL"],y_pred_elasticNet[0])[0])
#    print(name + "Correlation Original Data ",stats.pearsonr(y_train["RL"],pd.DataFrame(regr_elasticNet.predict(x_train))[0])[0])
    
    return  regr_elasticNet.score(x_test,y_test) ,stats.pearsonr(y_test.RL,pd.DataFrame(regr_elasticNet.predict(x_test))[0])[0]

## Function for Elastic NetCV  **** glmnet in R ***

In [12]:
def enet_cv(l1_ratio=0.5, eps=0.001, n_alphas=100, alphas=None, fit_intercept=True, normalize=False, precompute='auto',
            max_iter=1000, tol=0.0001, cv=None, copy_X=True, verbose=0, n_jobs=None, positive=False, random_state=None,
            selection='cyclic'):
    
    from sklearn.linear_model import ElasticNetCV
    global x_train,x_test,y_train, y_test , regr_enetcv , y_pred_elasticnetcv
    name = "ElasticnetCV --> "

    regr_enetcv = ElasticNetCV(l1_ratio=l1_ratio, eps=eps, n_alphas=n_alphas, alphas=alphas, fit_intercept=fit_intercept,
                               normalize=normalize,precompute=precompute, max_iter=max_iter, tol=tol, cv=cv,copy_X=copy_X,
                               verbose=verbose, n_jobs=n_jobs,positive=positive, random_state=random_state, selection=selection)
    
    regr_enetcv.fit(x_train,y_train)

    y_pred_elasticnetcv =pd.DataFrame(regr_enetcv.predict(x_test))
    
#    print(name +" Coefficient ", regr_enetcv.coef_) 
#    print(name + " Intercept " , regr_enetcv.intercept_,"\n") ##
#    
#    print("****"*5+"Accuracy Test Model Fitting "+"****"*5)
#    print(name + "Score for train data set :",regr_enetcv.score(x_train,y_train))
#    #Return the coefficient of determination R^2 of the prediction.
#    print(name + "Score for test data Set",regr_enetcv.score(x_test,y_test))
#    print(name + "Score for Predictecd data Set",regr_enetcv.score(x_test,y_pred_elasticnetcv)) ##
#    
#    #print("\n"+"****"*5+"R2 Score"+"****"*5)
#    #print(name + "R2 Score for test is = ", r2_score(y_test,y_pred_elasticnetcv)) ##
#    
#    print("\n"+"****"*5+"Mean Absolute Error"+"****"*5)
#    print(name+ "Mean Absolute Error of Test = ", mean_absolute_error(y_test,y_pred_elasticnetcv)) ##
#    print(name +"Mean Absolute Error on Train",mean_absolute_error(y_train,pd.DataFrame(regr_enetcv.predict(x_train))))
#    print("\n"+"****"*5+" Mean Squared Error"+"****"*5)
#    print(name + "Mean Squared Error of Test = ",mean_squared_error(y_test,y_pred_elasticnetcv)) ##
#    print(name+"Mean Squared Error on Train = ",mean_squared_error(y_train,pd.DataFrame(regr_enetcv.predict(x_train))))
#    print("\n"+"****"*5+" Co Relation"+"****"*5)
#    print(name + "Correlation Test Data ",stats.pearsonr(y_test["RL"],y_pred_elasticnetcv[0])[0])
#    print(name + "Correlation Original Data ",stats.pearsonr(y_train["RL"],pd.DataFrame(regr_enetcv.predict(x_train))[0])[0])
    
    return regr_enetcv.score(x_test,y_test) ,stats.pearsonr(y_test.RL,pd.DataFrame(regr_enetcv.predict(x_test))[0])[0]

## Function For Bayesian 

In [13]:
def bayesian_ridge(n_iter=300, tol=0.001, alpha_1=1e-06, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-06,
                compute_score=False, fit_intercept=True, normalize=False,copy_X=True, verbose=False):
 
    from sklearn.linear_model import BayesianRidge
    global x_train,x_test,y_train, y_test , regr_enetcv , y_pred_elasticnetcv,clf_bay , y_pred_bay
    
    name = "BayesianRidge"
    clf_bay = BayesianRidge(n_iter=n_iter, tol=tol, alpha_1=alpha_1, alpha_2=alpha_2, lambda_1=lambda_1, lambda_2=lambda_2,
                        compute_score=compute_score,fit_intercept=fit_intercept,normalize=normalize,copy_X=copy_X,
                            verbose=verbose).fit(x_train , y_train)
    
    y_pred_bay = pd.DataFrame(clf_bay.predict(x_test))
    
#    print(name +"Coefficient ",clf_bay.coef_) 
#    print(name + " Intercept " , clf_bay.intercept_,"\n") ##
#    print("****"*5+"Accuracy Test Model Fitting "+"****"*5)
#    print(name + "Score for train data set :",clf_bay.score(x_train,y_train))
    #Return the coefficient of determination R^2 of the prediction.
#    print(name + "Score for test data Set",clf_bay.score(x_test,y_test))
#    print(name + "Score for Predictecd data Set",clf_bay.score(x_test,y_pred_bay)) ##
    #print("\n"+"****"*5+"R2 Score"+"****"*5)
    #print(name + "R2 Score for test is = ", r2_score(y_test,y_pred_bay)) ##
#    print("\n"+"****"*5+"Mean Absolute Error"+"****"*5)
#    print(name+ "Mean Absolute Error of Test = ", mean_absolute_error(y_test,y_pred_bay)) ##
#    print(name +"Mean Absolute Error on Train",mean_absolute_error(y_train,pd.DataFrame(clf_bay.predict(x_train))))
#    print("\n"+"****"*5+" Mean Squared Error"+"****"*5)
#    print(name + "Mean Squared Error of Test = ",mean_squared_error(y_test,y_pred_bay)) ##
#    print(name+"Mean Squared Error on Train = ",mean_squared_error(y_train,pd.DataFrame(clf_bay.predict(x_train))))
#    print("\n"+"****"*5+" Co Relation"+"****"*5)
#    print(name + "Correlation Test Data ",stats.pearsonr(y_test["RL"],y_pred_bay[0])[0])
#    print(name + "Correlation Original Data ",stats.pearsonr(y_train["RL"],pd.DataFrame(clf_bay.predict(x_train))[0])[0])
    return clf_bay.score(x_test,y_test) ,stats.pearsonr(y_test.RL,pd.DataFrame(clf_bay.predict(x_test))[0])[0]

### Call different with different Parameter

In [14]:
train_test(size = 0.2 , random_state=5,shuffle =False)

### Call the Ridge Function in  with different Parameter

In [15]:
train_test(size = 0.2 , random_state=5,shuffle =False)
#***********************************************************
ridge(alpha=0.8,copy_X=True,fit_intercept=True, max_iter=10000, normalize=True)

(0.7445836649738835, 0.8725949100719352)

In [16]:
#alpha 0 is low result
#max_iter = 1000 is not influencing in the result
#Normalize False Reduce the Result
print(ridge(alpha=0.8,copy_X=True,fit_intercept=True, max_iter=1, normalize= False))

(0.7059587620026707, 0.8500134069667098)


In [ ]:
#Check with Suffle True 
#This also Reduce the Result so Best Ridge is 
#train_test(size = 0.2 , random_state=5,shuffle =True)
#ridge(alpha=0.8,copy_X=True,fit_intercept=True, max_iter=1, normalize=True)

#### Best Ridge Parameter

In [17]:
#train_test(size = 0.2 , random_state=5,shuffle =False)
#***********************************************************
ridge(alpha=0.8,copy_X=True,fit_intercept=True, max_iter=10, normalize=True)

(0.7445836649738835, 0.8725949100719352)

#### Bayesian For Testing

In [ ]:
train_test(size = 0.2 , random_state=5,shuffle = False)
#***********************************************************
bayesian_ridge(n_iter=300, tol=0.0001, alpha_1=1e-06, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-06,compute_score=True,
               fit_intercept=True, normalize=True,copy_X=True, verbose=False)

In [ ]:
#change Shuffle True low result
train_test(size = 0.2 , random_state=5,shuffle = True)
#***********************************************************
bayesian_ridge(n_iter=300, tol=0.0001, alpha_1=1e-06, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-06,compute_score=True,
               fit_intercept=True, normalize=True,copy_X=True, verbose=False)

In [ ]:
#change shuffle to False and increase n_iter to 1000,10000 not a very much different so we fixed to 1000

#train_test(size = 0.2 , random_state=5,shuffle = False)
#***********************************************************
bayesian_ridge(n_iter=10000, tol=0.0001, alpha_1=1e-06, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-06,compute_score=True,
               fit_intercept=True, normalize=True,copy_X=True, verbose=False)

### Call the   Lasso Regression with seprate Parameter

In [18]:
train_test(size=0.20,random_state=10, shuffle=True) #We are testing with same data  But different Parameter

In [19]:
#We have 2 Function  warm_state --> True
lasso( alpha=0, fit_intercept=True, normalize=False, precompute=False, copy_X=True,max_iter=1000,
      tol=0.0001,warm_start=True, positive=False,selection='random')

(0.6509064516377885, 0.8220546130757027)

In [20]:
#Change positive = True 
lasso(alpha=0, fit_intercept=True, normalize=False, precompute=False, copy_X=True,max_iter=1000,
      tol=0.0001,warm_start=False,positive=True,selection='cyclic')

(0.691175473142714, 0.8419249138871023)

In [21]:
#Change normalize = True Give low Result as Compared to above 0.6940006394147187, 0.8436065326019351)
#again change precompute = True also Give low result (0.6940006394147056, 0.8436065326019332)
# max_iter to 100 also low result
# max_iter to 10000 our data size Time Consuming but Result is little better in Test Data Set 
lasso(alpha=0, fit_intercept=True, normalize=False, precompute=False, copy_X=True,max_iter=10000,
      tol=0.0001,warm_start=False,positive=True,selection='cyclic')

(0.6977286137247007, 0.8439557448532988)

In [22]:
lasso(alpha=0, fit_intercept=True, normalize=False, precompute=False, copy_X=True,max_iter=1000,
      tol=0.0001,warm_start=False,positive=True,selection='random')

(0.6977958519291378, 0.8439773056709008)

In [ ]:
#We have 2 Function 
train_test(size=0.20,random_state=10, shuffle=True)
#***********************************************************
lasso( alpha=0, fit_intercept=True, normalize=False, precompute=False, copy_X=True,max_iter=1000, tol=0.00001,warm_start=False, positive=False,selection='cyclic')

In [ ]:
out = 0
for i in range(100):
    print(i*10)
    #We have 2 Function 
    #train_test(size=0.10,random_state=10, shuffle=True)
#***********************************************************
    a,b = lasso( alpha=0.01*i, fit_intercept=True, normalize=False, precompute=False,
          copy_X=True,max_iter=10, tol=0.0001,warm_start=False, positive=False,selection='cyclic')
    print(i,a,b)

## Elastic Net Regression  (Time Consuming)

In [ ]:
#train_test(size = 0.2 , random_state=580,shuffle = True)
#***********************************************************
enet(alpha=0, l1_ratio=0.01, fit_intercept=True, normalize=False,precompute=False, max_iter=1000,
         copy_X=True, tol=0.0001, warm_start=False, positive=False, random_state=None, selection='random')

## Bayesian Ridge Regression

In [23]:
train_test(size = 0.2 , random_state=580,shuffle = True)
#***********************************************************
bayesian_ridge(n_iter=100, tol=0.0001, alpha_1=1e-03, alpha_2=1e-03, lambda_1=1e-03, lambda_2=1e-03,compute_score=True,
               fit_intercept=True, normalize=True,copy_X=True, verbose=False)

(0.717292371464948, 0.8470718005445379)

In [ ]:
train_test(size = 0.20 , random_state=580,shuffle = True)
#***********************************************************
bayesian_ridge(n_iter=100, tol=0.0001, alpha_1=1e-06, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-06,compute_score=True,
               fit_intercept=True, normalize=True,copy_X=True, verbose=False)

In [ ]:
#Verbose True May take long time

train_test(size = 0.10 , random_state=580,shuffle = True)
#***********************************************************
bayesian_ridge(n_iter=100, tol=0.0001, alpha_1=1e-06, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-06,compute_score=True,
               fit_intercept=True, normalize=True,copy_X=True, verbose=True) #verbose True

In [ ]:
train_test(size = 0.10 , random_state=580,shuffle = True)
#***********************************************************
bayesian_ridge(n_iter=100, tol=0.0001, alpha_1=1e-06, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-06,compute_score=True,
               fit_intercept=True, normalize=True,copy_X=True, verbose=False) #verbose True

## Elastic Net CV ** Glmnet ** R
#### take long Time
## Reduce the sample Size while Testing this Method

In [ ]:
df.shape

In [35]:
train_test(size = 0.2 , random_state=580,shuffle = True)
##***********************************************************
enet_cv(l1_ratio=0.2, eps=0.001, n_alphas=10, alphas=None, fit_intercept=True, normalize=False, precompute=True,
            max_iter=1000, tol=0.0001, cv=10, copy_X=True, verbose=0, n_jobs=None, positive=False, random_state=None,
            selection='random')

(0.6938879452827396, 0.8454710218264059)

In [41]:
#Change l1_ration to 1
#Change selection random or cyclic
##***********************************************************
enet_cv(l1_ratio=0.2, eps=0.001, n_alphas=10, alphas=None, fit_intercept=True, normalize=False, precompute=True,
            max_iter=1000, tol=0.0001, cv=10, copy_X=True, verbose=0, n_jobs=None, positive=False, random_state=None,
            selection='cyclic') 

(0.6908473475323162, 0.8440305089982454)

In [42]:
#l1_ratio high reducing the Accuracy
#La_ratio reduce to 0.1
enet_cv(l1_ratio=0.1, eps=0.001, n_alphas=10, alphas=None, fit_intercept=True, normalize=True, precompute=True,
            max_iter=1000, tol=0.0001, cv=10, copy_X=True, verbose=0, n_jobs=None, positive=False, random_state=None,
            selection='random')

(0.706936659079988, 0.8505544306387687)

(8000, 3880)

### Testing Our new data with the Model developed 

In [24]:
y_pred_ridge.head()
test_x = pd.read_csv("../tsrl900.csv")
test_x = test_x.iloc[:,1:]

test_y = pd.read_csv("../ffyrgc900.csv")
test_y = test_y.iloc[:,1:]*100

In [45]:
print("Correlation with Ridge Reg Pred",stats.pearsonr(test_y["V1"],pd.DataFrame(clf_ridge.predict(test_x))[0])[0])
print("Correlation with Bay Reg Pred",stats.pearsonr(test_y["V1"],pd.DataFrame(clf_bay.predict(test_x))[0])[0])
print("Correlation with Lasso Pred",stats.pearsonr(test_y["V1"],pd.DataFrame(clf_lasso.predict(test_x))[0])[0])
print("Correlation with Enet CV ",stats.pearsonr(test_y["V1"],pd.DataFrame(regr_enetcv.predict(test_x))[0])[0])


Correlation with Ridge Reg Pred 0.8634330761372677
Correlation with Bay Reg Pred 0.8635354689164587
Correlation with Lasso Pred 0.8551005961414272
Correlation with Enet CV  0.8607536899428633


In [47]:
ridge_rl = pd.DataFrame(clf_ridge.predict(test_x))
bay_rl = pd.DataFrame(clf_bay.predict(test_x))
enet_rl = pd.DataFrame(regr_enetcv.predict(test_x))

In [29]:
test_y.head()

,V1
0,11.757538
1,12.110057
2,8.760571
3,8.592814
4,11.492376


In [51]:
ridge_rl.head()

,0
0,7.870367
1,11.901022
2,12.613933
3,8.762404
4,13.393478


## Result By Old Programming Concept

In [31]:
df_old = pd.read_csv("../corr_out_900",sep="\t",nrows=900)
df_old = df_old.iloc[:900,1:]
y_pred_old = df_old["Predicted FF"].astype(float)*100
y_test_old = df_old["Standard FF"].astype(float)*100

In [32]:
gc_pred_y = pd.read_csv("ridge_pred.csv",header = None)
new_pred = pd.DataFrame((gc_pred_y[0]+ridge_rl[0])/2)
gc_pred_y.head(2)

,0
0,11.703654
1,10.747002


In [33]:
#y_test_old.head() # Is same as test_y
#y_pred_old.head()
print("Co rrelation with New Model",stats.pearsonr(test_y["V1"],new_pred[0])[0])
print("Co relation with Old Model" ,stats.pearsonr(test_y["V1"],y_pred_old)[0])

print("\n"+"****"*5+"R2 Score"+"****"*5)
print("R2 of New Model " ,r2_score(test_y, new_pred))
print("R2 of Old Model " , r2_score(test_y, y_pred_old))

print("\n"+"****"*5+"Mean Absolute Error"+"****"*5)
print("Mean Absolute Error of Old = ", mean_absolute_error(test_y,y_pred_old))
print("Mean Absolute Error of New = ", mean_absolute_error(test_y,new_pred))

print("\n"+"****"*5+" Mean Squared Error"+"****"*5)
print("Mean Squared Error of Old = ",mean_squared_error(test_y,y_pred_old))
print("Mean Squared Error of New = ",mean_squared_error(test_y,new_pred))

Co rrelation with New Model 0.9179442588031144
Co relation with Old Model 0.9101507263893257

********************R2 Score********************
R2 of New Model  0.8042175253774206
R2 of Old Model  0.7998711754829443

********************Mean Absolute Error********************
Mean Absolute Error of Old =  1.3368891694449778
Mean Absolute Error of New =  1.3334588927342679

******************** Mean Squared Error********************
Mean Squared Error of Old =  3.2415920434758694
Mean Squared Error of New =  3.1711919236026014
